# 训练

## 准备数据集

### 读取数据集

读取我们之前步骤中准备好的数据集。

根据经验一般分类数据集都不大，为了方便以及运行效率，我们将它们一次性全部读入内存中。

如果爆内存了可以修改为要用哪些再读哪些。

### 图像变换和增强

神经网络是一个学习的过程，它会去学习训练集有什么“特征”。

假设我们猫的训练集全都是橘色的，神经网络可能会误以为橘色是“猫”的强特征；在实际推理时若有一张黑色的猫，它不符合“猫”的特征（不是橘色的），从而判断它不是猫。这种现象我们称之为过拟合。

最优的方案，当然是我们去拍摄更多猫的照片，其中也要包含各种颜色的猫。

凑合一点的方案，是我们将训练集复制几份，“P图”出各种颜色的猫。让神经网络知道，哦原来不管什么颜色都有可能是猫。

除了颜色，还有旋转、透视、缩放等等各种各样的“P”法，这一过程可以由框架自动完成，我们称之为图像的变换和增强，参考 [变换示例](https://pytorch.ac.cn/vision/stable/auto_examples/transforms/plot_transforms_illustrations.html#sphx-glr-auto-examples-transforms-plot-transforms-illustrations-py)。

更多的变换也需要更多的时间去计算，会增加训练的耗时，但是增加推理的鲁棒性。

所以需要根据实际应用场景来决定怎么变换。例如我要分类游戏中的角色，而每个角色的颜色都是固定的，自然就不需要颜色转换。或者如果地球上真的只有橘色的猫，也不需要。


In [1]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from pathlib import Path
from collections import defaultdict
from PIL import Image


class MyDataset(Dataset):
    def __init__(self, path: Path):
        super().__init__()

        self.transform = transforms.Compose(
            [
                transforms.GaussianBlur(3, sigma=(0.1, 2.0)), # 高斯模糊，卷积核的大小为3x3，水平和垂直方向上的标准偏差分别为 0.1 和 2.0
                transforms.RandomPosterize(3),                # 随机压缩
                transforms.RandomAdjustSharpness(3),          # 随机锐化
                transforms.RandomAutocontrast(),              # 自动对比度调整

                # ...... 更多变换请根据实际应用场景添加
                # https://pytorch.ac.cn/vision/stable/auto_examples/transforms/plot_transforms_illustrations.html#sphx-glr-auto-examples-transforms-plot-transforms-illustrations-py

                transforms.ToTensor(),  # 这个是一定要的
            ]
        )

        self.data = []
        for f in path.glob("*.png"):
            # 如果你的文件名不是这种格式，请自行修改此处代码
            label = f.stem.split("-")[0]

            # pytorch 实际会调用 __len__ 和 __getitem__
            # 如果内存不够，改为在 __getitem__ 再读文件即可
            image = Image.open(f).convert("RGB")

            self.data.append(image, label)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, label = self.data[idx]
        return (self.transform(image), label)


In [2]:
cwd = Path(".")
train_data = MyDataset(cwd / "data" / "train")
test_data = MyDataset(cwd / "data" / "test")
print(f"train len: {len(train_data)}")
print(f"test len: {len(test_data)}")

# 可根据内存/显存大小调整 batch_size，一般来说越大越快
train_loader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=0)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True, num_workers=0)

TypeError: list.append() takes exactly one argument (2 given)

## 构造神经网络

网络可以去 [SOTA](https://paperswithcode.com/task/image-classification) 里找：

它是按数据集跑分排行的，可以搜一下这些数据集是什么样的，找一个与你的任务最相似的。然后去里面再找一个准确率和体积比较合适的，抄一下它的网络。

针对 MaaFW 的开发场景，一般来说我们可能只有几百张软件的截图，不需要什么大型的网络，直接用 MNIST 里的或者网上找一写更简单的网络即可。


In [24]:
# 本次演示我们使用 GoogleNet，导入本目录下的 google_net.py
from google_net import GoogleNet
import torch

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
if not use_cuda:
    print("WARNING: CPU will be used for training.")

# channels - 输入图像的通道数。我们是 RGB 三通道
# linear - 线性层输入。这个要计算的，有点复杂，不用管，后面会报错，然后它会告诉你正确的是多少
# cls_count - 分类数。我们是 猫、狗、鸟 三分类
model = GoogleNet(channels=3, linear=1408, cls_count=3)

# 将数据移到 GPU/CPU
model = model.to(device)

## 构造损失函数和优化器

直接用 pytorch 现成的 API 即可，看不懂不用管，也不用管（我也不懂

In [25]:

from torch.amp import GradScaler

# 这里优化器用的 Adam，可以搜一下不同优化器的区别，挑一种合适的
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # 学习率可以自己调一下
criterion = torch.nn.CrossEntropyLoss().to(device)
scaler = GradScaler()

## 训练和测试

训练的主要三个步骤：前馈(forward) -> 反馈(backward) -> 更新(update)

同样的看不懂不用管，有兴趣可以找一下别的深度学习的教程

In [26]:
def train(epoch: int):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        # 将数据移到 GPU/CPU
        data = data.to(device)
        target = target.to(device)

        # 梯度归零
        optimizer.zero_grad()

        # 前馈
        ## 计算 y_pred
        output = model(data)
        ## 计算损失
        loss = criterion(output, target)

        # 反馈
        ## 反向传播
        scaler.scale(loss).backward()

        # 更新
        scaler.step(optimizer)
        scaler.update()

        print(
            f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx * len(data) / len(train_loader.dataset):.0f}%)]\tLoss: {loss.item():.6f}"
        )


def test():
    model.eval()
    test_loss = 0.0
    correct = 0
    # 测试不需要计算梯度
    with torch.no_grad():
        for data, target in test_loader:

            # 将数据移到 GPU/CPU
            data = data.to(device)
            target = target.to(device)

            # 前馈
            output = model(data)
            test_loss += criterion(output, target).item()

            # 计算准确率
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    acc = 100.0 * correct / len(test_loader.dataset)

    print(
        f"\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({acc:.0f}%)\n"
    )

    return test_loss, acc

## 简单的训练流程

训练一轮 -> 测一下这轮的损失和精确度 -> 如果比之前更优就保存一下

In [27]:
from pathlib import Path

output_dir = Path("checkpoints")
output_dir.mkdir(exist_ok=True, parents=True)

model_name = model.__class__.__name__
best_model_path = output_dir / f"{model_name}_best.pt"

best_epoch = 0
best_loss = 100.0
best_acc = 0.0


def pipeline(start_epoch=0):
    global best_epoch, best_loss, best_acc

    for epoch in range(start_epoch, 1000):
        train(epoch)
        loss, acc = test()

        torch.save(model, output_dir / f"{model_name}_{epoch}.pt")

        if loss > best_loss:
            if epoch - best_epoch > 100:
                print("No improvement for a long time, Early stop!")
                break
            else:
                continue
            
        best_epoch = epoch
        best_loss = loss
        best_acc = acc
        print(
            f"====== New best is {best_epoch}, Loss: {best_loss:.8f}, Acc: {best_acc:.4f} ======"
        )
        torch.save(model, best_model_path)


pipeline()


AttributeError: 'tuple' object has no attribute 'to'